# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [31]:
# Passo 1: Entrar na internet
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#Crome -> chromedriver
#Firefox -> geckodriver

navegador = webdriver.Chrome()


# Passo 2: Pegar a cotacao do dolar
# entrar no google
navegador.get("https://www.google.com/")
# pesquisar "cotacao dolar"
navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input'
).send_keys("cotacao dolar")
navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input'
).send_keys(Keys.ENTER)
    ## send_keys("Texto") == o que vai escrever
    ## send_keys(Keys.ENTER) == pressiona enter
# pegar o numero que aparece no resultado do google
cotacao_dolar = navegador.find_element_by_xpath(
'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]'
).get_attribute("data-value")
print(f'Dolar: {cotacao_dolar}')


# Passo 3: Pegar a cotacao do euro 
navegador.get("https://www.google.com/")
# pesquisar "cotacao euro"
navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input'
).send_keys("cotacao euro")
navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input'
).send_keys(Keys.ENTER)
    ## send_keys("Texto") == o que vai escrever
    ## send_keys(Keys.ENTER) == pressiona enter
# pegar o numero que aparece no resultado do google
cotacao_euro = navegador.find_element_by_xpath(
'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]'
).get_attribute("data-value")
print(f'Euro: {cotacao_euro}')


# Passo 4: Pegar a cotacao do ouro
# Entrar no site do melhor cambio
navegador.get("https://www.melhorcambio.com/ouro-hoje")
# pegar cotacao do ouro
cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]'
).get_attribute("value").replace(',', '.')
print(f'Ouro: {cotacao_ouro}')


navegador.quit()

Dolar: 5.2200999999999995
Euro: 6.1274055810000005
Ouro: 294.12


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [32]:
# Passo 5: Importar e atualizar a base de dados
import pandas as pd

tabela= pd.read_excel("Produtos.xlsx")
    # Se a tabela estiver na mesma pasta do arquivo, nao precisa passar 
    # o diretorio completo (C:\Users\User\...)
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [33]:
# Atualizar a cotacao
# Aonde ta a coluna Moeda = Dolar
# tabela.loc = localizar
# Sintaxe: tabela.loc[linha, coluna]

tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = round(float(cotacao_dolar), 2)

tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = round(float(cotacao_euro), 2)

tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = round(float(cotacao_ouro), 2)

# atualizar o preco de compra-> Preco Original * Cotacao
tabela["Preço de Compra"] = round(tabela["Preço Base Original"] * tabela["Cotação"], 2)

# atualizar o Preco de Venda -> Preco de Compra * Margem
tabela["Preço de Venda"] = round(tabela["Preço de Compra"] * tabela["Margem"], 2)
tabela["Preço Final"] = round(tabela["Preço Final"], 2)
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final,Preço de Compra,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.22,4999.95,1.40,6999.93,5219.95,7307.93
1,Carro Renault,4500.00,Euro,6.13,27000.00,2.00,54000.00,27585.00,55170.00
2,Notebook Dell,899.99,Dólar,5.22,4499.95,1.70,7649.91,4697.95,7986.52
3,IPhone,799.00,Dólar,5.22,3995.00,1.70,6791.50,4170.78,7090.33
4,Carro Fiat,3000.00,Euro,6.13,18000.00,1.90,34200.00,18390.00,34941.00
5,Celular Xiaomi,480.48,Dólar,5.22,2402.40,2.00,4804.80,2508.11,5016.22
6,Joia 20g,20.00,Ouro,294.12,7000.00,1.15,8050.00,5882.40,6764.76


### Agora vamos exportar a nova base de preços atualizada

In [34]:
# Passo 6: Exportar a base de dados atualizada
tabela.to_excel("Produtos Novo.xlsx", index=False)